In [ ]:
import pandas as pd
import numpy as np

In [ ]:
dataset =  pd.read_csv('course.csv', header=None)

In [ ]:
dataset.head()

In [ ]:
number_of_timeslot =int(dataset[0][0])
print(number_of_timeslot)

In [ ]:
number_of_rooms = dataset[1][0]
print(number_of_rooms)

In [ ]:
number_of_courses = dataset[1].nunique()-1
print(number_of_courses)

In [ ]:
code_of_courses = dataset[1].unique()
code_of_courses = code_of_courses[1:]
print(code_of_courses)

In [ ]:
number_of_student=dataset[0].nunique()
number_of_student

In [ ]:
dataset.dropna(inplace=True)

In [ ]:
conflict_matrix = pd.DataFrame(np.zeros(shape=(number_of_courses,number_of_courses)),dtype = int)

In [ ]:
conflict_matrix

In [ ]:
index_1 = 0;
index_2 = 0;
for course_1 in code_of_courses:
  number_of_student_within_course_1 = list(dataset[dataset[1] == course_1][0])
  index_2=0
  for course_2 in code_of_courses:
    number_of_student_within_course_2 = list(dataset[dataset[1] == course_2][0])
    conflict_matrix.at[index_1,index_2] = len(set(number_of_student_within_course_1)&set(number_of_student_within_course_2))
    index_2+=1
  index_1+=1

In [ ]:
 conflict_matrix

In [ ]:
conflict_matrix_array = conflict_matrix.to_numpy()
print(conflict_matrix_array)

In [ ]:
# timeslot_assignment = []
# for i in range(0,number_of_courses):
#   timeslot_assignment[i] = 0

timeslot_assignment = [0 for i in range(number_of_courses)]
print(len(timeslot_assignment))
print(timeslot_assignment)

In [ ]:
def is_available_slot(courseid, timeslot):
  if timeslot not in timeslot_assignment:
    return True
  elif timeslot in timeslot_assignment:
    course_slot = [idx for idx, course in enumerate(timeslot_assignment) if course == timeslot]
    ls = []
    for adjacent_course in course_slot:
      ls.append(conflict_matrix_array[courseid][adjacent_course])
    if any(ls) > 0:
      return False
    else:
      return True

In [ ]:
for courses_index in range(0, number_of_courses):
  for timeslot in range(1,number_of_timeslot+1):
    if is_available_slot(courses_index, timeslot):
      timeslot_assignment[courses_index] = timeslot
      break

In [ ]:
def is_feasible(solution):
  length_of_timeslot = len(solution)
  score = 0
  for i in range(length_of_timeslot - 1):
    for j in range(length_of_timeslot - 1):
      j+=i+1
      if j > length_of_timeslot-1:
        break
      if conflict_matrix_array[i][j] != 0 and solution[i]== solution[j]:
        return False
    

  return True
print(is_feasible(timeslot_assignment))

In [ ]:
def objective_function(solution):
  length_of_timeslot = len(solution)
  score = 0
  for i in range(length_of_timeslot - 1):
    for j in range(length_of_timeslot - 1):
      j+=i+1
      if j > length_of_timeslot-1:
        break
      if solution[j] - solution[i]>=3:
        score += 0
      else:
        score += 2**(solution[j]-solution[i]) * conflict_matrix_array[j][i]
  return score/number_of_student



In [ ]:
of = objective_function(timeslot_assignment)
print(of)

In [ ]:
import random
def create_new_solution(solution):
  loop = False
  while not loop:
    random_course = random.randint(0, number_of_courses-1) 
    old_timeslot = solution[random_course]
    random_timeslot = random.randint(1, number_of_timeslot)
    solution[random_course] = random_timeslot
    loop = is_feasible(solution)
    if not loop:
      solution[random_course] = old_timeslot     
  return solution.copy()

In [ ]:
num_of_iteration = 1000
for i in range(num_of_iteration):
  new_timeslot_assignment = create_new_solution(timeslot_assignment.copy())
  if objective_function(new_timeslot_assignment) < objective_function(timeslot_assignment):
    timeslot_assignment = new_timeslot_assignment.copy()


In [ ]:
print(objective_function(timeslot_assignment))

In [ ]:
print(timeslot_assignment)